In [298]:
import pandas as pd
import numpy as np

import os, json

In [299]:
import spacy
nlp = spacy.load('en_core_web_sm')

## Read rss data

In [301]:
# Read Json files
path_to_json = 'rssData'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
#print(json_files)

In [252]:
# Go through Json files 
base_dir = 'rssData'

#Get all files in the directory

data_list = []
for file in os.listdir(base_dir):

    #If file is a json, construct it's full path and open it, append all json data to list
    if 'json' in file:
        json_path = os.path.join(base_dir, file)
        json_data = pd.read_json(json_path, lines=True)
        data_list.append(json_data)

#print(data_list)

In [302]:
len(data_list)

198

In [305]:
type(data_list)

list

In [306]:
type(data_list[0])

pandas.core.frame.DataFrame

In [254]:
data_list[0].text

0    
Name: text, dtype: object

In [255]:
data_list[0].text.values

array([''], dtype=object)

In [256]:
data_list[7].header_title.values

array(['A Retrospective on the 2015 Ashley Madison Breach'], dtype=object)

In [257]:
data_list[2].header_title.values

array(['911 Proxy Service Implodes After Disclosing Breach'], dtype=object)

In [258]:
# Get text data and remove empty texts
all_text=[]
for i in range(len(data_list)):
    if (data_list[i].text.values!=''):
        text=list(data_list[i].text)
#         print(type(text))
        all_text.append(text)
        #print(data_list[i].text)
        #print(text)
#print(all_text)

In [259]:
len(all_text)

147

In [260]:
all_text_df= pd.DataFrame(all_text, columns=['text'])
all_text_df.head()

,text
0,"The 911 service as it existed until July 28, 2..."
1,"DanielMiessler Created/Updated: July 25, 2022 ..."
2,The 911 service as it exists today.For the pas...
3,DanielMiesslerMy first thought on the whole di...
4,"DanielMiesslerWell, our congressional heroes f..."


In [297]:
data_list[7].text.values

array(['It’s been seven years since the online cheating site AshleyMadison.com was hacked and highly sensitive data about its users posted online. The leak led to the public shaming and extortion of many Ashley Madison users, and to at least two suicides. To date, little is publicly known about the perpetrators or the true motivation for the attack. But a recent review of Ashley Madison mentions across Russian cybercrime forums and far-right websites in the months leading up to the hack revealed some previously unreported details that may deserve further scrutiny.As first reported by KrebsOnSecurity on July 19, 2015, a group calling itself the “Impact Team” released data sampled from millions of users, as well as maps of internal company servers, employee network account information, company bank details and salary information.The Impact Team said it decided to publish the information because ALM “profits on the pain of others,” and in response to a paid “full delete” service Ashley Ma

In [261]:
all_text_df['text'][5]

'It’s been seven years since the online cheating site AshleyMadison.com was hacked and highly sensitive data about its users posted online. The leak led to the public shaming and extortion of many Ashley Madison users, and to at least two suicides. To date, little is publicly known about the perpetrators or the true motivation for the attack. But a recent review of Ashley Madison mentions across Russian cybercrime forums and far-right websites in the months leading up to the hack revealed some previously unreported details that may deserve further scrutiny.As first reported by KrebsOnSecurity on July 19, 2015, a group calling itself the “Impact Team” released data sampled from millions of users, as well as maps of internal company servers, employee network account information, company bank details and salary information.The Impact Team said it decided to publish the information because ALM “profits on the pain of others,” and in response to a paid “full delete” service Ashley Madison p

In [262]:
# function to clean data
#lower case
#remove stop words
#lemmatization

def cleanData(doc):
    doc = doc.lower()
    doc = nlp(doc)
    tokens = [tokens.lower_ for tokens in doc]
    tokens = [tokens for tokens in doc if (tokens.is_stop == False)]
    tokens = [tokens for tokens in tokens if (tokens.is_punct == False)]
    final_token = [token.lemma_ for token in tokens]
    
    return " ".join(final_token)

In [263]:
all_text_df['clean'] = all_text_df.apply(lambda row:cleanData (row['text']),axis=1)

In [264]:
all_text_df.head()

,text,clean
0,"The 911 service as it existed until July 28, 2...",911 service exist july 28 2022.911[.]re proxy ...
1,"DanielMiessler Created/Updated: July 25, 2022 ...",danielmiessler create update july 25 2022 read...
2,The 911 service as it exists today.For the pas...,911 service exist today.for past seven year on...
3,DanielMiesslerMy first thought on the whole di...,danielmiesslermy think discussion sure musk ar...
4,"DanielMiesslerWell, our congressional heroes f...",danielmiesslerwell congressional hero finally ...


## Implement NLP techniques

https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.Y-Zq4XbMI2w

### Calculate IDF

In [265]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer

In [266]:
# CountVectorizer() 
cv=CountVectorizer() 
# this steps generates word counts for the words in all docs 
word_count_vector=cv.fit_transform(all_text_df['clean'])

In [267]:
word_count_vector.shape

(147, 8382)

In [268]:
# Compute idf weights
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
all_text_df_tfidf = tfidf_transformer.fit_transform(word_count_vector)
all_text_df_tfidf.shape

(147, 8382)

In [269]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names_out(),columns=["idf_weights"]) 
# sort ascending 
df_idf.sort_values(by=['idf_weights'],ascending=False).head()

,idf_weights
earli,5.304065
resolved,5.304065
george,5.304065
geopolitical,5.304065
geometry,5.304065


<font color=green> The higher idf_weights means most unique word in any particular doc</font>

### Calculate TF-IDF

In [270]:
# count matrix 
count_vector=cv.transform(all_text_df['clean']) 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [271]:
feature_names = cv.get_feature_names_out() 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[5] 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df=df.sort_values(by=["tfidf"],ascending=False)

In [272]:
df.head(10)

,tfidf
ashley,0.361024
madison,0.345327
nazi,0.275439
alm,0.204057
biderman,0.172664
hack,0.147875
profile,0.146008
team,0.122921
2015,0.119916
impact,0.116995


<font color=green>the more unique a word is to our first document, the higher the score

In [273]:
def POS(clean):
    clean = nlp(clean)
    for token in clean:
        return(token.pos_) 

# Word Extraction Based on TF-IDF

###  doc #5

### Title : 'A Retrospective on the 2015 Ashley Madison Breach'

In [274]:
# data_list[7].header_title.values

In [275]:
# Most frequent words
df = df.reset_index(names=['word', 'tfidf'])
df['pos'] = df.apply(lambda row:POS (row['word']),axis=1)

In [276]:
df1 = df[(df['word'].str.len()) > 3]

In [277]:
print('Most Frequent Words Based on TF-IDF:')
df1.head(10)

Most Frequent Words Based on TF-IDF:


,word,tfidf,pos
0,ashley,0.361024,PROPN
1,madison,0.345327,PROPN
2,nazi,0.275439,ADV
4,biderman,0.172664,NOUN
5,hack,0.147875,VERB
6,profile,0.146008,NOUN
7,team,0.122921,NOUN
8,2015,0.119916,NUM
9,impact,0.116995,NOUN
11,user,0.103592,NOUN


In [278]:
print('Most frequent words based on NOUN and VERB')
df1[(df1['pos'] == 'NOUN') | (df1['pos'] == 'VERB')].head(10)[['word','tfidf']]

Most frequent words based on NOUN and VERB


,word,tfidf
4,biderman,0.172664
5,hack,0.147875
6,profile,0.146008
7,team,0.122921
9,impact,0.116995
11,user,0.103592
15,company,0.089976
16,leak,0.087470
17,promise,0.083925
20,adultery,0.078483


In [279]:
print('Most frequent words Based on NOUN')
df1[df1['pos']=='NOUN'].head(10)[['word','tfidf']]

Most frequent words Based on NOUN


,word,tfidf
4,biderman,0.172664
6,profile,0.146008
7,team,0.122921
9,impact,0.116995
11,user,0.103592
15,company,0.089976
16,leak,0.087470
20,adultery,0.078483
21,female,0.074673
25,email,0.065374


In [280]:
print('Most frequent words Based on VERB')
df1[df1['pos']=='VERB'].head(10)[['word','tfidf']]

Most frequent words Based on VERB


,word,tfidf
5,hack,0.147875
17,promise,0.083925
22,mention,0.073554
29,duckduckgo,0.062787
40,delete,0.057971
41,lead,0.057555
45,believe,0.052429
49,imagine,0.048636
60,think,0.045050
61,remove,0.044949


### Named Entity Recognition (NER)

In [216]:
#function to NER 
# Iterate over the predicted entities
def NER(clean):
    clean = nlp(clean)
    for ent in clean.ents:
        return(ent.label_)   

In [217]:
NER(df1['word'][2])

'NORP'

In [218]:
df1['ner_tag'] = df1.apply(lambda row:NER (row['word']),axis=1)

C:\Users\msalehi\AppData\Local\Temp\ipykernel_16396\542725880.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['ner_tag'] = df1.apply(lambda row:NER (row['word']),axis=1)


In [219]:
df1.head(10)

,word,tfidf,pos,ner_tag
0,ashley,0.361024,PROPN,None
1,madison,0.345327,PROPN,PERSON
2,nazi,0.275439,ADV,NORP
4,biderman,0.172664,NOUN,None
5,hack,0.147875,VERB,None
6,profile,0.146008,NOUN,None
7,team,0.122921,NOUN,None
8,2015,0.119916,NUM,DATE
9,impact,0.116995,NOUN,None
11,user,0.103592,NOUN,None


In [220]:
df1[(df1['pos'] == 'NOUN') | (df1['pos'] == 'VERB')].head(10)[['word','tfidf','pos','ner_tag']]

,word,tfidf,pos,ner_tag
4,biderman,0.172664,NOUN,None
5,hack,0.147875,VERB,None
6,profile,0.146008,NOUN,None
7,team,0.122921,NOUN,None
9,impact,0.116995,NOUN,None
11,user,0.103592,NOUN,None
15,company,0.089976,NOUN,None
16,leak,0.087470,NOUN,None
17,promise,0.083925,VERB,None
20,adultery,0.078483,NOUN,None
